In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-16 15:09:56.503731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 15:09:56.567067: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-16 15:09:56.569087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-16 15:09:56.569097: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [21]:
def ffnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
    model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    checkpoint = ModelCheckpoint('../models/ffnn_checkpoint.h5', monitor='val_loss', save_best_only=True,
                                save_weights_only=False)
    early_stopping = EarlyStopping(monitor='val_loss', patience=300)
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=[checkpoint, early_stopping],
          validation_split=0.2, 
         )
    best_model = keras.models.load_model('../models/ffnn_checkpoint.h5')
    return best_model
#     return model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

#### delete from here

In [8]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [9]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [10]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [11]:
# X_train[0]

In [12]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [13]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [14]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [15]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [16]:
# mean_squared_error(y_test, y_pred)

In [17]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [18]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train, y_train)
    y_pred = model.predict(X_test) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

#### delete from here

In [22]:
idx=0
device_indices = final_df[final_df.latitude==latitudes[idx]].index
device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
test_df = final_df.loc[device_indices]
train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)

In [23]:
# %%timeit
model = ffnn(X_train, y_train)

Epoch 1/1000
289/289 [==============================] - 1s 1ms/step - loss: 96044528.0000 - val_loss: 1578.1554
Epoch 2/1000
289/289 [==============================] - 0s 1ms/step - loss: 32200.2227 - val_loss: 1565.7485
Epoch 3/1000
289/289 [==============================] - 0s 1ms/step - loss: 8392.7725 - val_loss: 1551.1180
Epoch 4/1000
289/289 [==============================] - 0s 899us/step - loss: 5714.4663 - val_loss: 1535.6742
Epoch 5/1000
289/289 [==============================] - 0s 943us/step - loss: 11159.9082 - val_loss: 1522.3210
Epoch 6/1000
289/289 [==============================] - 0s 1ms/step - loss: 8234.6895 - val_loss: 1507.4973
Epoch 7/1000
289/289 [==============================] - 0s 1ms/step - loss: 3450.0317 - val_loss: 1491.8115
Epoch 8/1000
289/289 [==============================] - 0s 1ms/step - loss: 997.2740 - val_loss: 1475.4044
Epoch 9/1000
289/289 [==============================] - 0s 992us/step - loss: 2047.6837 - val_loss: 1459.7737
Epoch 10/1000
289

Epoch 76/1000
289/289 [==============================] - 0s 1ms/step - loss: 2167.2141 - val_loss: 774.5467
Epoch 77/1000
289/289 [==============================] - 0s 971us/step - loss: 9937.6162 - val_loss: 770.6545
Epoch 78/1000
289/289 [==============================] - 0s 1ms/step - loss: 953.7294 - val_loss: 765.6997
Epoch 79/1000
289/289 [==============================] - 0s 965us/step - loss: 1629.9756 - val_loss: 761.1424
Epoch 80/1000
289/289 [==============================] - 0s 1ms/step - loss: 3536.6772 - val_loss: 756.7839
Epoch 81/1000
289/289 [==============================] - 0s 917us/step - loss: 2670.0579 - val_loss: 752.7768
Epoch 82/1000
289/289 [==============================] - 0s 1ms/step - loss: 1188.2843 - val_loss: 748.5137
Epoch 83/1000
289/289 [==============================] - 0s 1ms/step - loss: 1568.5486 - val_loss: 744.4105
Epoch 84/1000
289/289 [==============================] - 0s 1ms/step - loss: 425.9561 - val_loss: 740.1152
Epoch 85/1000
289/289 [=

289/289 [==============================] - 0s 1ms/step - loss: 421.0814 - val_loss: 667.7689
Epoch 152/1000
289/289 [==============================] - 0s 939us/step - loss: 430.9088 - val_loss: 667.7184
Epoch 153/1000
289/289 [==============================] - 0s 962us/step - loss: 553.7941 - val_loss: 667.5731
Epoch 154/1000
289/289 [==============================] - 0s 990us/step - loss: 1680.0826 - val_loss: 667.3939
Epoch 155/1000
289/289 [==============================] - 0s 994us/step - loss: 562.3708 - val_loss: 667.3033
Epoch 156/1000
289/289 [==============================] - 0s 962us/step - loss: 411.6691 - val_loss: 667.3176
Epoch 157/1000
289/289 [==============================] - 0s 1ms/step - loss: 2831.2717 - val_loss: 667.1094
Epoch 158/1000
289/289 [==============================] - 0s 1ms/step - loss: 578.7726 - val_loss: 667.0937
Epoch 159/1000
289/289 [==============================] - 0s 1ms/step - loss: 463.4751 - val_loss: 667.1383
Epoch 160/1000
289/289 [=======

289/289 [==============================] - 0s 964us/step - loss: 1696.6873 - val_loss: 665.7922
Epoch 226/1000
289/289 [==============================] - 0s 937us/step - loss: 643.6166 - val_loss: 665.8380
Epoch 227/1000
289/289 [==============================] - 0s 1ms/step - loss: 538.6564 - val_loss: 665.8537
Epoch 228/1000
289/289 [==============================] - 0s 963us/step - loss: 2037.7422 - val_loss: 665.8858
Epoch 229/1000
289/289 [==============================] - 0s 943us/step - loss: 568.2186 - val_loss: 666.0014
Epoch 230/1000
289/289 [==============================] - 0s 898us/step - loss: 966.4988 - val_loss: 666.0676
Epoch 231/1000
289/289 [==============================] - 0s 884us/step - loss: 427.4627 - val_loss: 666.1246
Epoch 232/1000
289/289 [==============================] - 0s 998us/step - loss: 472.5540 - val_loss: 666.2498
Epoch 233/1000
289/289 [==============================] - 0s 930us/step - loss: 3666.3237 - val_loss: 666.1536
Epoch 234/1000
289/289 [

Epoch 300/1000
289/289 [==============================] - 0s 1ms/step - loss: 432.0098 - val_loss: 668.7585
Epoch 301/1000
289/289 [==============================] - 0s 920us/step - loss: 411.8080 - val_loss: 668.8379
Epoch 302/1000
289/289 [==============================] - 0s 851us/step - loss: 501.8224 - val_loss: 668.8775
Epoch 303/1000
289/289 [==============================] - 0s 940us/step - loss: 411.5946 - val_loss: 668.9645
Epoch 304/1000
289/289 [==============================] - 0s 1000us/step - loss: 411.8009 - val_loss: 669.0710
Epoch 305/1000
289/289 [==============================] - 0s 971us/step - loss: 411.5971 - val_loss: 669.0911
Epoch 306/1000
289/289 [==============================] - 0s 1ms/step - loss: 411.6233 - val_loss: 669.1072
Epoch 307/1000
289/289 [==============================] - 0s 997us/step - loss: 412.5599 - val_loss: 669.1435
Epoch 308/1000
289/289 [==============================] - 0s 900us/step - loss: 451.6877 - val_loss: 669.1095
Epoch 309/100

Epoch 375/1000
289/289 [==============================] - 0s 890us/step - loss: 544.3140 - val_loss: 668.1556
Epoch 376/1000
289/289 [==============================] - 0s 961us/step - loss: 5549.1670 - val_loss: 668.0028
Epoch 377/1000
289/289 [==============================] - 0s 1ms/step - loss: 516.2723 - val_loss: 668.2044
Epoch 378/1000
289/289 [==============================] - 0s 981us/step - loss: 411.6552 - val_loss: 668.2450
Epoch 379/1000
289/289 [==============================] - 0s 946us/step - loss: 411.7033 - val_loss: 668.2892
Epoch 380/1000
289/289 [==============================] - 0s 905us/step - loss: 411.5820 - val_loss: 668.3617
Epoch 381/1000
289/289 [==============================] - 0s 975us/step - loss: 411.3758 - val_loss: 668.4288
Epoch 382/1000
289/289 [==============================] - 0s 961us/step - loss: 2165.8911 - val_loss: 668.2906
Epoch 383/1000
289/289 [==============================] - 0s 968us/step - loss: 414.8547 - val_loss: 668.2818
Epoch 384/

289/289 [==============================] - 0s 1ms/step - loss: 411.8664 - val_loss: 668.4664
Epoch 451/1000
289/289 [==============================] - 0s 1ms/step - loss: 411.7216 - val_loss: 668.4052
Epoch 452/1000
289/289 [==============================] - 0s 986us/step - loss: 412.4218 - val_loss: 668.2900
Epoch 453/1000
289/289 [==============================] - 0s 1ms/step - loss: 411.6971 - val_loss: 668.2688
Epoch 454/1000
289/289 [==============================] - 0s 971us/step - loss: 459.4941 - val_loss: 668.2108
Epoch 455/1000
289/289 [==============================] - 0s 970us/step - loss: 1461.1821 - val_loss: 668.1588
Epoch 456/1000
289/289 [==============================] - 0s 917us/step - loss: 411.6878 - val_loss: 668.1424
Epoch 457/1000
289/289 [==============================] - 0s 974us/step - loss: 411.7689 - val_loss: 668.1232
Epoch 458/1000
289/289 [==============================] - 0s 988us/step - loss: 419.3241 - val_loss: 668.0963
Epoch 459/1000
289/289 [======

In [26]:
# %%timeit
y_pred = model.predict(X_test) 

66/66 [==============================] - 0s 676us/step


In [27]:
y_pred.shape

(2104, 1)

#### end here

In [ ]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

In [ ]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

In [ ]:
rmse_list

In [ ]:
mean_mape = np.mean(mape_list)          
mean_mape

In [ ]:
mape_list